In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [56]:
ceased_labours = pd.read_json('https://www.dati.lombardia.it/resource/nwz3-p6vm.json')
ceased_labours.head()

,data,genere,eta,settoreeconomicodettaglio,titolostudio,contratto,modalitalavoro,provinciaimpresa,italiano
0,2019-04-02T00:00:00.000,M,20,Lavori di meccanica generale,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,APPRENDISTATO PROFESSIONALIZZANTE O CONTRATTO ...,TEMPO PIENO,BERGAMO,ITALIA
1,2018-06-08T00:00:00.000,F,28,Fabbricazione di altre apparecchiature elettri...,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,APPRENDISTATO PROFESSIONALIZZANTE O CONTRATTO ...,TEMPO PIENO,BERGAMO,ITALIA
2,2016-02-02T00:00:00.000,F,20,Fabbricazione di altre apparecchiature elettri...,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,APPRENDISTATO PROFESSIONALIZZANTE O CONTRATTO ...,TEMPO PIENO,BERGAMO,ITALIA
3,2015-01-12T00:00:00.000,M,59,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,ITALIA
4,2017-08-09T00:00:00.000,M,61,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,ITALIA


In [57]:
ceased_labours.loc[:, ['settoreeconomicodettaglio']].value_counts()

settoreeconomicodettaglio                                                      
Ristorazione con somministrazione                                                  108
Costruzione di edifici residenziali e non residenziali                              57
Attività di famiglie e convivenze come datori di lavoro per personale domestico     53
Fabbricazione di altri rubinetti e valvole                                          46
Bar e altri esercizi simili senza cucina                                            41
                                                                                  ... 
Commercio di energia elettrica                                                       1
Commercio all'ingrosso di tappeti                                                    1
Commercio all'ingrosso di parti e accessori di autoveicoli                           1
Commercio all'ingrosso di carne fresca, congelata e surgelata                        1
Villaggi turistici                                

In [58]:
ceased_labours.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   data                       1000 non-null   object
 1   genere                     1000 non-null   object
 2   eta                        1000 non-null   int64 
 3   settoreeconomicodettaglio  993 non-null    object
 4   titolostudio               1000 non-null   object
 5   contratto                  1000 non-null   object
 6   modalitalavoro             1000 non-null   object
 7   provinciaimpresa           1000 non-null   object
 8   italiano                   1000 non-null   object
dtypes: int64(1), object(8)
memory usage: 70.4+ KB


In [59]:
ceased_labours.iloc[:, 0].values

array(['2019-04-02T00:00:00.000', '2018-06-08T00:00:00.000',
       '2016-02-02T00:00:00.000', '2015-01-12T00:00:00.000',
       '2017-08-09T00:00:00.000', '2012-10-12T00:00:00.000',
       '2013-03-04T00:00:00.000', '2021-06-10T00:00:00.000',
       '2010-09-04T00:00:00.000', '2020-01-07T00:00:00.000',
       '2015-10-03T00:00:00.000', '2013-02-01T00:00:00.000',
       '2017-02-11T00:00:00.000', '2019-04-05T00:00:00.000',
       '2020-11-06T00:00:00.000', '2013-02-05T00:00:00.000',
       '2017-10-11T00:00:00.000', '2013-03-09T00:00:00.000',
       '2013-03-09T00:00:00.000', '2016-11-02T00:00:00.000',
       '2019-04-05T00:00:00.000', '2020-02-05T00:00:00.000',
       '2011-11-04T00:00:00.000', '2010-09-04T00:00:00.000',
       '2011-04-04T00:00:00.000', '2015-04-12T00:00:00.000',
       '2009-03-06T00:00:00.000', '2013-03-01T00:00:00.000',
       '2011-10-10T00:00:00.000', '2014-06-06T00:00:00.000',
       '2011-09-09T00:00:00.000', '2013-09-10T00:00:00.000',
       '2018-06-11T00:00

In [60]:
ceased_labours['year'] = ceased_labours['data'].apply(lambda x:x[0: 4])
ceased_labours['year'].value_counts()

2019    159
2020    129
2021    128
2018     90
2011     65
2013     62
2010     59
2017     57
2012     53
2016     52
2015     52
2014     49
2009     44
2008      1
Name: year, dtype: int64

In [62]:
ceased_labours[ceased_labours.year == '2019']

,data,genere,eta,settoreeconomicodettaglio,titolostudio,contratto,modalitalavoro,provinciaimpresa,italiano,year
0,2019-04-02T00:00:00.000,M,20,Lavori di meccanica generale,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,APPRENDISTATO PROFESSIONALIZZANTE O CONTRATTO ...,TEMPO PIENO,BERGAMO,ITALIA,2019
13,2019-04-05T00:00:00.000,F,53,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,2019
20,2019-04-05T00:00:00.000,M,49,"Gestioni di funicolari, ski-lift e seggiovie s...",LICENZA MEDIA,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,BERGAMO,ITALIA,2019
39,2019-04-01T00:00:00.000,F,24,"Discoteche, sale da ballo night-club e simili",LICENZA MEDIA,LAVORO INTERMITTENTE,NON DEFINITO,LECCO,ITALIA,2019
44,2019-06-06T00:00:00.000,F,25,Attività di musei,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,LAVORO INTERMITTENTE,NON DEFINITO,LECCO,ITALIA,2019
...,...,...,...,...,...,...,...,...,...,...
961,2019-02-01T00:00:00.000,M,31,Trasporto di merci su strada,LICENZA MEDIA,LAVORO A TEMPO INDETERMINATO,TEMPO PIENO,MANTOVA,MAROCCO,2019
964,2019-09-10T00:00:00.000,M,21,Alberghi,DIPLOMA DI ISTRUZIONE SECONDARIA SUPERIORE CH...,TIROCINIO,TEMPO PARZIALE ORIZZONTALE,MANTOVA,ITALIA,2019
968,2019-03-01T00:00:00.000,F,80,Attività di famiglie e convivenze come datori ...,NESSUN TITOLO DI STUDIO,LAVORO DOMESTICO,TEMPO PARZIALE ORIZZONTALE,MANTOVA,ITALIA,2019
978,2019-10-06T00:00:00.000,M,40,Altre attività connesse ai trasporti terrestri...,NESSUN TITOLO DI STUDIO,LAVORO A TEMPO DETERMINATO,TEMPO PIENO,MANTOVA,ITALIA,2019
